# 1.2 Locate the name of the asset

I also tried to locate the asset name of the sample dataset so that I can find the real data to fill the dataset’s missing values. Hinted from the description “a well-known broad market ETF”, I first try collect the data for the top ETFs from website (https://stockmarketmba.com/listoftop100etfs.php) saved as csv file named “List of Top 100 ETFs.xlsx”. Only thing we need is column A - symbols. 

In [1]:
import yfinance as yf
import numpy as np
import pandas as pd

In [2]:
ETF_list = pd.read_excel('List of Top 100 ETFs.xlsx', usecols = ['Symbol'])
ETF_list = np.array(ETF_list['Symbol'].tolist())
ETF_list[:10]

array(['SPY', 'IVV', 'VTI', 'VOO', 'QQQ', 'VEA', 'IEFA', 'AGG', 'VWO',
       'VTV'], dtype='<U4')

Below downloads the historical prices of the 100 ETFs from 1Jan2015 to 31Jan2021, which covers the sample data's dates, saved as csv file named “Prices for 100ETFs 19Nov2015-06Jan2020.csv”

In [3]:
ETFdata = pd.DataFrame()
start_date = '2015-11-01'
end_date = '2020-01-31'

In [4]:
for ticker in ETF_list:
    ticker_holder = yf.Ticker(ticker)
    temp_data = ticker_holder.history(start = start_date,end = end_date,
                                      auto_adjust = False)
    temp_data = temp_data[['Close']]
    temp_data = temp_data.rename({'Close': ticker}, axis='columns')
    ETFdata = pd.concat([ETFdata,temp_data], axis = 1, join = 'outer')

In [5]:
data = ETFdata['2015-11-19':'2020-01-06']

In [6]:
data.iloc[:,:10].head()

,SPY,IVV,VTI,VOO,QQQ,VEA,IEFA,AGG,VWO,VTV
Date,,,,,,,,,,
2015-11-19,208.550003,209.669998,106.760002,191.220001,113.709999,38.009998,56.209999,108.690002,34.919998,83.110001
2015-11-20,209.309998,210.619995,107.220001,191.929993,114.480003,37.930000,56.070000,108.620003,35.220001,83.169998
2015-11-23,209.070007,210.220001,107.110001,191.710007,114.150002,37.700001,55.730000,108.650002,34.930000,83.019997
2015-11-24,209.350006,210.520004,107.389999,192.020004,114.050003,37.750000,55.720001,108.750000,35.020000,83.250000
2015-11-25,209.320007,210.529999,107.470001,191.960007,114.150002,37.810001,55.840000,108.800003,34.750000,83.150002


In [7]:
data.to_csv('Prices for 100ETFs 19Nov2015-06Jan2020.csv')

Then need to find if there is a match in this 100 ETFs.

In [8]:
sample = pd.read_csv('Sample Dataset_corrected.csv')
sample.loc[:,'Date']=pd.to_datetime(sample['Date'])
sample.set_index('Date',inplace = True)
sample_close = sample[['Close']]
sample_close = sample_close.rename({'Close':'Sample'}, axis='columns')
sample_close.head()

,Sample
Date,
2015-11-19,116.059998
2015-11-20,116.809998
2015-11-23,117.389999
2015-11-24,118.250000
2015-11-25,119.169998


In [9]:
new_data = pd.concat([data,sample_close], axis = 1, join = 'outer')
new_data.iloc[:,-5:].head()

,IWS,MINT,XLC,IEF,Sample
Date,,,,,
2015-11-19,70.970001,100.849998,NaN,106.089996,116.059998
2015-11-20,71.129997,100.860001,NaN,105.949997,116.809998
2015-11-23,71.209999,100.839996,NaN,106.089996,117.389999
2015-11-24,71.510002,100.860001,NaN,106.190002,118.250000
2015-11-25,71.500000,100.870003,NaN,106.250000,119.169998


In [10]:
diff_frame = new_data.sub(new_data['Sample'].tolist(),axis = 'index')
diff_frame.iloc[:,-5:].head()

,IWS,MINT,XLC,IEF,Sample
Date,,,,,
2015-11-19,-45.089997,-15.210000,NaN,-9.970002,0.0
2015-11-20,-45.680001,-15.949997,NaN,-10.860001,0.0
2015-11-23,-46.180000,-16.550003,NaN,-11.300003,0.0
2015-11-24,-46.739998,-17.389999,NaN,-12.059998,0.0
2015-11-25,-47.669998,-18.299995,NaN,-12.919998,0.0


In [11]:
diff_series = (np.round(diff_frame - 0, decimals=5)==0).sum(axis = 'index')
diff_series[diff_series>0]

IWM       1019
VO           2
IWB          1
Sample    1038
dtype: int64

In [12]:
#get IWM full data
ticker = 'IWM'
a = yf.Ticker(ticker)
iwm = a.history(start=start_date, end = end_date, auto_adjust = False)
iwm = iwm['2015-11-19':'2020-01-06']
iwm.head()

,Open,High,Low,Close,Adj Close,Volume,Dividends,Stock Splits
Date,,,,,,,,
2015-11-19,116.440002,116.650002,115.739998,116.059998,107.580505,25512500,0.0,0
2015-11-20,116.480003,117.360001,116.379997,116.809998,108.275726,31697700,0.0,0
2015-11-23,116.709999,117.889999,116.680000,117.389999,108.813324,22716400,0.0,0
2015-11-24,116.879997,118.419998,116.559998,118.250000,109.610512,24994500,0.0,0
2015-11-25,118.300003,119.320000,118.110001,119.169998,110.463310,20772600,0.0,0


In [13]:
diff_df = iwm.loc[:,'Open':'Close'] - sample.loc[:,'Open':'Close']
diff_df.round(5).sum()

Open     -5.15999
High      0.46997
Low      18.41996
Close    32.28998
dtype: float64

So **IWM is the underlying asset**. I will use it to evaluate the efficacy of the signals.

In [14]:
iwm.to_csv('IWM 19Nov2015-06Jan2020.csv')